In [1]:
import glob
import torch

from data_utils import *

In [2]:
data_dir = '../sum_data/'
file_vocab = 'cnn_vocab_abs.txt'
file_corpus = 'cnn_abs.txt'
batch_size = 64

vocab2id, id2vocab = construct_vocab(data_dir+'/'+file_vocab)
print 'The vocabulary size: {0}'.format(len(vocab2id))

The vocabulary size: 22682


In [23]:
def process_minibatch(batch_id, vocab2id, max_lens=[100, 20]):
    
    folder = 'batch_folder'
    fkey = 'batch_'
    file_ = folder + '/' + fkey + str(batch_id)
    fp = open(file_, 'r')
    src_arr = []
    trg_arr = []
    src_lens = []
    trg_lens = []
    for line in fp:
        arr = re.split('<sec>', line[:-1])

        dabs = re.split('\s', arr[0])
        dabs = filter(None, dabs)
        dabs = ['<s>'] + dabs + ['</s>']
        trg_lens.append(len(dabs))
        dabs2id = [
            vocab2id[wd] if wd in vocab2id
            else vocab2id['<unk>']
            for wd in dabs
        ]
        trg_arr.append(dabs2id)
        
        dart = re.split('\s', arr[1])
        dart = filter(None, dart)
        dart = ['<s>'] + dart + ['</s>']
        src_lens.append(len(dart))
        dart2id = [
            vocab2id[wd] if wd in vocab2id
            else vocab2id['<unk>']
            for wd in dart
        ]
        src_arr.append(dart2id)
    fp.close()
    
    src_max_lens = max(src_lens) 
    if max_lens[0] < max(src_lens):
        src_max_lens = max_lens[0]
    trg_max_lens = max(trg_lens)
    if max_lens[1] < max(trg_lens):
        trg_max_lens = max(trg_lens)
        
    print src_max_lens, trg_max_lens
    
    src_arr = [itm[:max_lens[0]] for itm in src_arr]
    trg_arr = [itm[:max_lens[1]] for itm in trg_arr]

    src_arr = [
        itm[:-1] + [vocab2id['<pad>']]*(1+src_max_lens-len(itm))
        for itm in src_arr
    ]
    trg_input_arr = [
        itm[:-1] + [vocab2id['<pad>']]*(1+trg_max_lens-len(itm))
        for itm in trg_arr
    ]
    trg_output_arr = [
        itm[1:] + [vocab2id['<pad>']]*(1+trg_max_lens-len(itm))
        for itm in trg_arr
    ]
    
    src_var = Variable(torch.LongTensor(src_arr))
    trg_input_var = Variable(torch.LongTensor(trg_input_arr))
    trg_output_var = Variable(torch.LongTensor(trg_output_arr))
    
    return src_var, trg_input_var, trg_output_var

process_minibatch(100, vocab2id)

81 19


(Variable containing:
      0    275      9  ...       2      2      2
      0  21688      3  ...       2      2      2
      0     23     13  ...       2      2      2
         ...            ⋱           ...         
      0     23     13  ...       2      2      2
      0    599   9139  ...       2      2      2
      0      3  20048  ...       2      2      2
 [torch.LongTensor of size 64x81], Variable containing:
      0   4266   6866  ...       2      2      2
      0  18107  14786  ...       2      2      2
      0    468   3462  ...       2      2      2
         ...            ⋱           ...         
      0  12395    241  ...       2      2      2
      0   1977   5316  ...       2      2      2
      0    893   6963  ...       2      2      2
 [torch.LongTensor of size 64x19], Variable containing:
   4266   6866  14226  ...       2      2      2
  18107  14786    429  ...       2      2      2
    468   3462     21  ...       2      2      2
         ...            ⋱        

In [25]:
fp = open('../sum_data/cnn_abs.txt')
len = 0
for line in fp:
    arr = re.split('<sec>', line[:-1])
    print arr[0]
    print arr[1]
    break
fp.close()

 woman portrayed as a teen in 'the sound of music ' has died 
 agathe von trapp was oldest daughter of family portrayed in the `` the sound of music '' .  'she was nothing at all like the liesl character , ' her friend says .  four von trapp children surviv .
